In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from scipy import optimize

%matplotlib inline 
import matplotlib.pyplot as plt

In [2]:
from simul_functions import S_i_fit

In [3]:
mean_S=[114.64866107858613, 281.263665030094, 434.1710447166398, 569.9988028399946, 687.0546718639996, 780.4870062669543, 864.119024879455, 916.7825679232428, 975.9915880272431, 1022.9939908123263]
rms=[114.74968236197647, 282.01041012696396, 434.47275315291114, 570.8325673130487, 687.4374797608367, 781.6874291625296, 864.5162056661065, 917.0785397347968, 976.4170138865792, 1024.7237580132255]
sigma_S=[4.813950047724185, 20.509074913742488, 16.188797420809703, 30.84128185045552, 22.93831837303278, 43.304387297040904, 26.20268499031733, 23.297450379212293, 28.82022045794128, 59.515334146904]
S_ref_values=[524.36037518,717.49813895,850.49419746, 715.4684653,951.54946654,893.63784202,811.12501887,896.93268679,870.97201082,1003.64425617]

In [4]:
b=0.919
a=-1.13
c=1

In [5]:
def lnlike(theta, x, y, yerr):
    a,b,c,S_ref_values,lnf=theta
    model = S_ref_values*(a * x**2 + b*x+c)
    inv_sigma2 = 1.0/(yerr**2 + model**2*np.exp(2*lnf))
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))

In [7]:
x=np.linspace(0.05,0.95,10, endpoint = True)
y=S_i_fit(x,a,b,S_ref_values)
print(type(y))

<class 'numpy.ndarray'>


In [25]:
import scipy.optimize as op
N=len(mean_S)

for i in range(0,N):
    nll = lambda *args: -lnlike(*args)
    x0=[a, b, c, S_ref_values[i], np.log(np.asarray(mean_S)[i]) ]
    result= op.minimize(nll, np.asarray(x0), args=(x, y, np.asarray(sigma_S)[i]))
    a_ml, b_ml,c_ml,S_ref_ml, lnf_ml = result["x"]

In [27]:
print(result["x"])

[-6.96439986e-01  1.87318840e+00 -2.82229277e-02  1.00409425e+03
 -9.84069528e+00]


In [29]:
import scipy.optimize as op
N=len(mean_S)
result=[]
a_ml=[]
for i in range(0,N):
    nll = lambda *args: -lnlike(*args)
    x0=[a, b, c, S_ref_values[i], np.log(np.asarray(mean_S)[i]) ]
    result.append( op.minimize(nll, np.asarray(x0), args=(x, y, np.asarray(sigma_S)[i])))
    a_ml.append(result[0])
    #a_ml, b_ml,c_ml,S_ref_ml, lnf_ml = result["x"]

In [30]:
print(result)
#print(a_ml)

[      fun: 42.009752997557264
 hess_inv: array([[ 9.96093493e-02, -7.16887818e-02,  4.20735600e-03,
         2.73866753e-02,  1.78585220e-04],
       [-7.16887818e-02,  5.82561631e-02, -3.67709982e-03,
        -2.25693850e-02, -1.89091682e-03],
       [ 4.20735600e-03, -3.67709982e-03,  3.65016820e-04,
         7.73570756e-04,  1.25932443e-04],
       [ 2.73866753e-02, -2.25693850e-02,  7.73570756e-04,
         1.01399087e+00,  2.43753574e-02],
       [ 1.78585220e-04, -1.89091682e-03,  1.25932443e-04,
         2.43753574e-02,  5.57089979e-02]])
      jac: array([1.43051147e-06, 9.53674316e-07, 4.76837158e-07, 0.00000000e+00,
       9.53674316e-07])
  message: 'Optimization terminated successfully.'
     nfev: 420
      nit: 27
     njev: 60
   status: 0
  success: True
        x: array([-1.48390191e+00,  3.73577381e+00, -7.74610984e-02,  5.23925588e+02,
       -2.58658231e+00]),       fun: 67.76489342384303
 hess_inv: array([[ 3.43344483e-06, -6.05606438e-06,  4.04162459e-07,
       